<a href="https://colab.research.google.com/github/laribar/bitcoinprediction/blob/main/modelo_yfinance_geral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para baixar dados e calcular métricas técnicas
def baixar_dados(acao, intervalo="1d", periodo="2y"):
    try:
        df = yf.download(acao, period=periodo, interval=intervalo, progress=False, auto_adjust=False)
        if df.empty or df.isnull().values.any():
            logging.error(f"Erro: Dados faltantes ou inválidos para {acao}.")
            return None

        preco_coluna = "Adj Close" if "Adj Close" in df.columns else "Close"

        if preco_coluna not in df.columns:
            logging.error(f"Erro ao processar {acao}: Nenhuma coluna de fechamento encontrada nos dados.")
            return None

        # Calcula a média móvel simples (SMA)
        df["SMA"] = df[preco_coluna].rolling(window=20).mean()

        # Calcula o desvio padrão da janela
        df["STD"] = df[preco_coluna].rolling(window=20).std()

        # Calcula as bandas superior e inferior corretamente
        df["Upper_Band"] = df["SMA"] + (df["STD"] * 2)
        df["Lower_Band"] = df["SMA"] - (df["STD"] * 2)

        # Remove colunas auxiliares que podem estar causando erro
        df = df.drop(columns=["STD"])

        # Calcula suporte e resistência
        df["Support"] = df["Low"].rolling(window=20).min()
        df["Resistance"] = df["High"].rolling(window=20).max()

        # Calcula LTAs e LTBs
        df["LTA"] = df["High"].rolling(window=10).max()  # Linha de Tendência de Alta (máximos)
        df["LTB"] = df["Low"].rolling(window=10).min()   # Linha de Tendência de Baixa (mínimos)

        return df

    except Exception as e:
        logging.error(f"Erro ao baixar dados para {acao}: {e}", exc_info=True)
        return None

# Função para treinar o modelo e prever preços futuros
def prever_precos(df, acao, intervalo):
    try:
        # Selecionar features e target
        X = df[['SMA', 'Upper_Band', 'Lower_Band', 'Support', 'Resistance', 'LTA', 'LTB']]
        y = df['Close'].values.ravel()  # Converter y para array 1D

        # Definir cutoff_date como 98% dos dados
        cutoff_index = int(0.98 * len(df))  # Índice numérico para corte
        X_train, X_test = X.iloc[:cutoff_index], X.iloc[cutoff_index:]
        y_train, y_test = y[:cutoff_index], y[cutoff_index:]

        # Verificar se há dados suficientes para treinamento e teste
        if len(X_train) == 0 or len(X_test) == 0:
            logging.error(f"Erro: Dados insuficientes para treinamento/teste em {acao} no intervalo {intervalo}")
            return None

        # Padronizar os dados
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Ajuste de hiperparâmetros com GridSearchCV
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10]
        }
        model = RandomForestRegressor(random_state=42)
        grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error')
        grid_search.fit(X_train_scaled, y_train)

        best_model = grid_search.best_estimator_

        # Criar datas futuras
        future_dates = [df.index[-1] + pd.Timedelta(days=i) for i in range(1, 6)]

        # Criar features futuras com base em tendências
        future_features = pd.DataFrame({
            'SMA': [df['SMA'].iloc[-1] + i * 0.1 for i in range(5)],
            'Upper_Band': [df['Upper_Band'].iloc[-1] + i * 0.1 for i in range(5)],
            'Lower_Band': [df['Lower_Band'].iloc[-1] + i * 0.1 for i in range(5)],
            'Support': [df['Support'].iloc[-1] + i * 0.1 for i in range(5)],
            'Resistance': [df['Resistance'].iloc[-1] + i * 0.1 for i in range(5)],
            'LTA': [df['LTA'].iloc[-1] + i * 0.1 for i in range(5)],
            'LTB': [df['LTB'].iloc[-1] + i * 0.1 for i in range(5)]
        }, index=future_dates)

        future_features_scaled = scaler.transform(future_features)

        # Fazer previsões
        future_predictions = best_model.predict(future_features_scaled)
        return future_predictions

    except Exception as e:
        logging.error(f"Erro ao prever preços para {acao}: {e}", exc_info=True)
        return None

# Função para gerar alertas de compra/venda
def gerar_alertas(df, previsoes, acao, intervalo):
    try:
        # Garantir que last_price seja um valor escalar
        last_price = df['Close'].iloc[-1]
        if isinstance(last_price, pd.Series):
            last_price = last_price.item()  # Converte para um valor escalar

        if previsoes is not None and len(previsoes) > 0:
            avg_future_price = np.mean(previsoes)

            # Garantir que todas as métricas sejam valores escalares
            sma = df['SMA'].iloc[-1].item() if isinstance(df['SMA'].iloc[-1], pd.Series) else df['SMA'].iloc[-1]
            upper_band = df['Upper_Band'].iloc[-1].item() if isinstance(df['Upper_Band'].iloc[-1], pd.Series) else df['Upper_Band'].iloc[-1]
            lower_band = df['Lower_Band'].iloc[-1].item() if isinstance(df['Lower_Band'].iloc[-1], pd.Series) else df['Lower_Band'].iloc[-1]
            support = df['Support'].iloc[-1].item() if isinstance(df['Support'].iloc[-1], pd.Series) else df['Support'].iloc[-1]
            resistance = df['Resistance'].iloc[-1].item() if isinstance(df['Resistance'].iloc[-1], pd.Series) else df['Resistance'].iloc[-1]
            lta = df['LTA'].iloc[-1].item() if isinstance(df['LTA'].iloc[-1], pd.Series) else df['LTA'].iloc[-1]
            ltb = df['LTB'].iloc[-1].item() if isinstance(df['LTB'].iloc[-1], pd.Series) else df['LTB'].iloc[-1]

            # Definir ordens de compra e venda
            ordem_compra = max(support, lower_band)  # Ordem de compra: máximo entre suporte e banda inferior
            ordem_venda = min(resistance, upper_band)  # Ordem de venda: mínimo entre resistência e banda superior

            # Gerar alerta e justificativa
            if avg_future_price > last_price * 1.02:  # Tendência de alta
                alerta = '🚀 Compra sugerida'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) é 2% maior que o preço atual ({last_price:.2f}). "
                    f"Além disso, a SMA está em {sma:.2f}, a banda superior em {upper_band:.2f}, e a banda inferior em {lower_band:.2f}. "
                    f"O suporte está em {support:.2f}, a resistência em {resistance:.2f}, e a LTA em {lta:.2f}. "
                    f"Isso indica uma tendência de alta."
                )
            elif avg_future_price < last_price * 0.98:  # Tendência de baixa
                alerta = '⚠️ Venda sugerida'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) é 2% menor que o preço atual ({last_price:.2f}). "
                    f"Além disso, a SMA está em {sma:.2f}, a banda superior em {upper_band:.2f}, e a banda inferior em {lower_band:.2f}. "
                    f"O suporte está em {support:.2f}, a resistência em {resistance:.2f}, e a LTB em {ltb:.2f}. "
                    f"Isso indica uma tendência de baixa."
                )
            else:
                alerta = '🔍 Acompanhar'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) está dentro de 2% do preço atual ({last_price:.2f}). "
                    f"A SMA está em {sma:.2f}, a banda superior em {upper_band:.2f}, e a banda inferior em {lower_band:.2f}. "
                    f"O suporte está em {support:.2f}, a resistência em {resistance:.2f}, a LTA em {lta:.2f}, e a LTB em {ltb:.2f}. "
                    f"Isso sugere que o mercado está estável no momento."
                )
        else:
            alerta = '❌ Sem previsões disponíveis'
            justificativa = 'Não há previsões disponíveis para esta ação.'
            ordem_compra = None
            ordem_venda = None

        # Exibir resultados
        print(f"Stock: {acao} | Intervalo: {intervalo}")
        print(f"Last price: {last_price:.2f}")
        print(f"Predictions: {previsoes}")
        print(f"Alert: {alerta}")
        print(f"Ordem de Compra: {ordem_compra:.2f}" if ordem_compra else "Ordem de Compra: N/A")
        print(f"Ordem de Venda: {ordem_venda:.2f}" if ordem_venda else "Ordem de Venda: N/A")
        print(f"Justificativa: {justificativa}")
        print("-" * 80)

    except Exception as e:
        logging.error(f"Erro ao gerar alertas para {acao}: {e}", exc_info=True)

# Função para obter o Top 10 da NASDAQ por capitalização de mercado
def obter_top_10_nasdaq():
    """
    Obtém o Top 10 da NASDAQ por capitalização de mercado.
    """
    # Baixar a lista de tickers da NASDAQ
    nasdaq_tickers = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
    top_10 = nasdaq_tickers.head(10)['Ticker'].tolist()
    return top_10

# Processar ações
acoes = obter_top_10_nasdaq()
for acao in acoes:
    df = baixar_dados(acao)
    if df is not None:
        previsoes = prever_precos(df, acao, "1d")
        gerar_alertas(df, previsoes, acao, "1d")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: ADBE | Intervalo: 1d
Last price: 389.61
Predictions: [427.82700134 427.25700134 427.25700134 427.25700134 445.70000122]
Alert: 🚀 Compra sugerida
Ordem de Compra: 376.36
Ordem de Venda: 453.75
Justificativa: A média das previsões futuras (431.06) é 2% maior que o preço atual (389.61). Além disso, a SMA está em 426.48, a banda superior em 476.61, e a banda inferior em 376.36. O suporte está em 374.50, a resistência em 453.75, e a LTA em 452.51. Isso indica uma tendência de alta.
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: AMD | Intervalo: 1d
Last price: 107.14
Predictions: [103.99387932 103.62627021 102.62579575 102.10209707 101.91867036]
Alert: ⚠️ Venda sugerida
Ordem de Compra: 94.73
Ordem de Venda: 109.94
Justificativa: A média das previsões futuras (102.85) é 2% menor que o preço atual (107.14). Além disso, a SMA está em 102.08, a banda superior em 109.94, e a banda inferior em 94.22. O suporte está em 94.73, a resistência em 115.76, e a LTB em 94.73. Isso indica uma tendência de baixa.
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: AMGN | Intervalo: 1d
Last price: 315.04
Predictions: [331.73057777 331.69241116 331.69241116 331.74083791 331.75975451]
Alert: 🚀 Compra sugerida
Ordem de Compra: 302.02
Ordem de Venda: 326.24
Justificativa: A média das previsões futuras (331.72) é 2% maior que o preço atual (315.04). Além disso, a SMA está em 314.13, a banda superior em 326.24, e a banda inferior em 302.02. O suporte está em 297.19, a resistência em 335.88, e a LTA em 335.88. Isso indica uma tendência de alta.
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: ADI | Intervalo: 1d
Last price: 205.90
Predictions: [212.43297201 212.40009221 212.13904954 212.13904954 212.08294122]
Alert: 🚀 Compra sugerida
Ordem de Compra: 202.59
Ordem de Venda: 242.97
Justificativa: A média das previsões futuras (212.24) é 2% maior que o preço atual (205.90). Além disso, a SMA está em 220.29, a banda superior em 242.97, e a banda inferior em 197.61. O suporte está em 202.59, a resistência em 246.08, e a LTA em 226.66. Isso indica uma tendência de alta.
--------------------------------------------------------------------------------
Stock: ANSS | Intervalo: 1d
Last price: 324.63
Predictions: [322.5353278  322.54035685 323.4086028  324.01742927 323.97451491]
Alert: 🔍 Acompanhar
Ordem de Compra: 315.23
Ordem de Venda: 336.23
Justificativa: A média das previsões futuras (323.30) está dentro de 2% do preço atual (324.63). A SMA está em 325.73, a banda superior em 336.23, e a banda inferior em 315.23. O suporte está em 314.11, a resistência em 340.31, a LTA em 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
